# Ecuaciones Hiperbólicas

El objetivo es observar como se compartan los diferentes esquemas vistos en la Práctica 3. 

Consideremos el problema $$U_t+aU_x=0$$

con

$$U(x,0)=U_0(x)=\left\{\begin{array}{rr}
1 & 0.1< x <0.4 \\
0 & x<0.4
\end{array}\right.
$$

Recordar que en este caso la solución de la ecucación estará dada por $$U(x,t)=U_0(x-at)$$

In [1]:
using Plots

In [3]:
function U₀(x)
    if 0.1<x<=0.4
        return 1
    else
        return 0
    end
end

function U(x,t;a=0.9)
    return U₀(x-a*t)
end

U (generic function with 1 method)

#### Método Up-Wind

Sea $\nu=\frac{\Delta t}{\Delta x}$ el método up-wind usando backward queda determinado por $$u^{n+1}_j=u^n_j-a\nu(u^n_j-u^n_{j-1})$$

In [44]:
function up_wind(U₀,U,Δx,Δt,steps;a=0.9)

    x = 0:Δx:3
    u = zeros(length(x),steps)
    for j=1:length(x)
        u[j,1]=U₀(x[j])
    end

    ν=Δt/Δx
    for n=2:steps
        for j=2:length(x)
            u[j,n]=u[j,n-1]-ν*a*(u[j,n-1]-u[j-1,n-1])
        end
    end

    U_ex=zeros(length(x),steps)
    for n=1:steps
        for j=1:length(x)
            U_ex[j,n]=U(x[j],Δt*n)
        end
    end
    #plot(x,U_ex[:,steps],ls=:dash,label="Solución exacta")
    #plot!(x,u[:,steps],label="Solución numérica")
    return x,u,U_ex
end

up_wind (generic function with 2 methods)

In [46]:
up_wind(U₀,U,0.01,0.005,400)

(0.0:0.01:3.0, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 6.189221693782113e-16 9.754890719888996e-16; 0.0 0.0 … 2.6848786705810757e-16 4.2618330310215424e-16], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [ ]:
Δx = 0.01
steps=500
x,u,U_ex = up_wind(U₀,U,Δx,0.005,steps)
anim = @animate for n=1:steps
    plot(x,U_ex[:,n],ls=:dash,label="Solución exacta")
    plot!(x,u[:,n],label="Solución numérica")
end
gif(anim, "up_wind.gif", fps = 15) 

#### Método Lax-Wendroff

Sea $\nu=\frac{\Delta t}{\Delta x}$, el método up-wind usando backward queda determinado por $$u^{n+1}_j=u^n_j-\frac{a}{2}\nu(u^n_{j+1}-u^n_{j-1})+\frac{a^2}{2}\nu^2(u^n_{j+1}-2u^n_j+u^n_{j-1})$$ 

In [34]:
function lax_wendroff(U₀,U,Δx,Δt,steps;a=0.9)

    x=0:Δx:3
    u = zeros(length(x),steps)
    for j=1:length(x)
        u[j,1]=U₀(x[j])
    end

    ν=Δt/Δx
    for n=2:steps
        for j=2:length(x)-1
            u[j,n]=u[j,n-1]-ν*(a/2)*(u[j+1,n-1]-u[j-1,n-1])-(a^2)*(0.5*Δx^2)*Δt*(u[j+1,n-1]-2*u[j,n-1]+u[j-1,n-1])
        end
    end

    U_ex=zeros(length(x),steps)
    for n=1:steps
        for j=1:length(x)
            U_ex[j,n]=U(x[j],Δt*n)
        end
    end

    #plot(x,U_ex[:,steps],ls=:dash,label="Solución exacta")
    #plot!(x,u[:,steps],label="Solución numérica")
    return x,u,U_ex
end

lax_wendroff (generic function with 2 methods)

In [36]:
lax_wendroff(U₀,U,0.02,0.001,5)

(0.0:0.02:3.0, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [ ]:
Δx = 0.01
steps=100
x,u,U_ex = lax_wendroff(U₀,U,Δx,0.001,steps)
anim = @animate for n=1:steps
    plot(x,U_ex[:,n],ls=:dash,label="Solución exacta")
    plot!(x,u[:,n],label="Solución numérica")
end
gif(anim, "lax_wendroff.gif", fps = 5) 

**Comentario:** Las oscilaciones surgen porque el esquema de Lax-Wendroff
no satisface un principio de máximo. Vemos en $$ u^{n+1}_j = \frac{1}{2}\nu(1 + \nu)u^n_{j-1} + (1 - \nu^2)u^n_j - \frac{1}{2} \nu(1 - \nu)u^n_{j+1}$$ que con
$\nu > 0$ el coeficiente de $u^n_{j+1}$ es negativo, ya que requerimos que $\nu ≤ 1$ para la estabilidad. Por lo tanto $u^{n+1}_j$ puede verse como un promedio ponderado de tres valores en el nivel de tiempo anterior, pero dos de los pesos son positivos y uno es negativo. Por lo tanto, es posible que la solución numérica tenga oscilaciones con máximos y mínimos internos.

Como ejemplo de un problema con una solución suave, consideramos la misma ecuación que antes, pero reemplazamos la condición inicial por $$U(x,0)=e^{-10(4x-1)^2}$$

Realizar el mismo analisis de antes y notar que ahora el pulso tiene una forma gaussiana suave, en lugar de una onda cuadrada discontinua. Al utilizar los mismos tamaños de malla que antes, los resultados son considerablemente más precisos. Todavía hay algún signo de oscilación a la izquierda del pulso en el momento, pero es mucho menor que en el caso discontinuo. Además, el uso del tamaño de malla más pequeño ha reducido el tamaño de los errores y esta oscilación se vuelve casi invisible.

In [3]:
#completar

In [1]:
#Completar

#### **Ejercicio**:
Implementar los esquemas de Lax-Friedrichs y Leapfrog realizar el mismo análisis.

In [38]:
#completar